In [1]:
pip install tqdm

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import gym
import numpy as np
from tqdm import tqdm

# Hyperparameters
gamma = 0.99
epsilon = 0.1
c1 = 0.5
c2 = 0.01
clip_range = 0.2
max_episodes = 1000
max_steps = 500
batch_size = 32
actor_lr = 0.001
critic_lr = 0.001

# Environment
env = gym.make('CartPole-v1')

# Actor and Critic Networks
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize Actor and Critic
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
actor = Actor(state_dim, action_dim)
critic = Critic(state_dim)

# Optimizers
actor_optimizer = optim.Adam(actor.parameters(), lr=actor_lr)
critic_optimizer = optim.Adam(critic.parameters(), lr=critic_lr)

# Experience Buffer
class ExperienceBuffer:
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.dones = []

    def add_experience(self, state, action, reward, next_state, done):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.next_states.append(next_state)
        self.dones.append(done)

    def sample_experience(self, batch_size):
        indices = np.random.choice(len(self.states), batch_size)
        states = np.array(self.states)[indices]
        actions = np.array(self.actions)[indices]
        rewards = np.array(self.rewards)[indices]
        next_states = np.array(self.next_states)[indices]
        dones = np.array(self.dones)[indices]
        return states, actions, rewards, next_states, dones

# Training Loop
experience_buffer = ExperienceBuffer()
for episode in tqdm(range(max_episodes)):
    state = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        # Select action
        state_tensor = torch.tensor(state, dtype=torch.float32)
        action_logits = actor(state_tensor)
        action_distribution = Categorical(logits=action_logits)
        action = action_distribution.sample().item()

        # Take action
        next_state, reward, done, _ = env.step(action)
        episode_reward += reward

        # Add experience to buffer
        experience_buffer.add_experience(state, action, reward, next_state, done)

        # Update state
        state = next_state

        # Check if episode is done
        if done:
            break

    # Sample experience from buffer
    states, actions, rewards, next_states, dones = experience_buffer.sample_experience(batch_size)

    # Compute advantages
    advantages = []
    for i in range(batch_size):
        state_tensor = torch.tensor(states[i], dtype=torch.float32)
        next_state_tensor = torch.tensor(next_states[i], dtype=torch.float32)
        reward = rewards[i]
        done = dones[i]
        value = critic(state_tensor).item()
        next_value = critic(next_state_tensor).item()
        advantage = reward + gamma * next_value * (1 - done) - value
        advantages.append(advantage)
    advantages = np.array(advantages)

    # Compute clipped surrogate objective
    old_action_logits = actor(torch.tensor(states, dtype=torch.float32)).detach()
    old_action_distribution = Categorical(logits=old_action_logits)
    old_action_log_probs = old_action_distribution.log_prob(torch.tensor(actions))
    new_action_logits = actor(torch.tensor(states, dtype=torch.float32))
    new_action_distribution = Categorical(logits=new_action_logits)
    new_action_log_probs = new_action_distribution.log_prob(torch.tensor(actions))
    ratio = torch.exp(new_action_log_probs - old_action_log_probs)
    clipped_ratio = torch.clamp(ratio, 1 - clip_range, 1 + clip_range)
    clipped_surrogate_objective = torch.mean(torch.minimum(ratio * torch.tensor(advantages), clipped_ratio * torch.tensor(advantages)))

    # Compute value loss
    value_loss = torch.mean((critic(torch.tensor(states, dtype=torch.float32)) - torch.tensor(rewards + gamma * critic(torch.tensor(next_states, dtype=torch.float32)).detach() * (1 - torch.tensor(dones)))) ** 2)

    # Compute entropy loss
    entropy_loss = torch.mean(new_action_distribution.entropy())

    # Compute total loss
    total_loss = -clipped_surrogate_objective + c1 * value_loss - c2 * entropy_loss

    # Backpropagate and update
    actor_optimizer.zero_grad()
    critic_optimizer.zero_grad()
    total_loss.backward()
    actor_optimizer.step()
    critic_optimizer.step()

    # Print episode reward
    print(f'Episode {episode+1}, Reward: {episode_reward}')

# Test the trained agent
state = env.reset()
episode_reward = 0
while True:
    state_tensor = torch.tensor(state, dtype=torch.float32)
    action_logits = actor(state_tensor)
    action_distribution = Categorical(logits=action_logits)
    action = action_distribution.sample().item()
    next_state, reward, done, _ = env.step(action)
    episode_reward += reward
    state = next_state
    if done:
        break
print(f'Test Episode Reward: {episode_reward}')

/usr/local/lib/python3.10/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.10/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
  0%|          | 0/1000 [00:00<?, ?it/s]


RuntimeError: Subtraction, the `-` operator, with a bool tensor is not supported. If you are trying to invert a mask, use the `~` or `logical_not()` operator instead.

In [5]:
!pip install gymnasium

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 958.1/958.1 kB 15.8 MB/s eta 0:00:00


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.distributions import Categorical
import gymnasium as gym
import numpy as np
from tqdm import tqdm

# Hyperparameters
gamma = 0.99
epsilon = 0.1
c1 = 0.5
c2 = 0.01
clip_range = 0.2
max_episodes = 1000
max_steps = 500
batch_size = 32
actor_lr = 0.001
critic_lr = 0.001

# Environment
env = gym.make('CartPole-v1', render_mode='human')

# Actor and Critic Networks
class Actor(nn.Module):
    def __init__(self, state_dim, action_dim):
        super(Actor, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, action_dim)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

class Critic(nn.Module):
    def __init__(self, state_dim):
        super(Critic, self).__init__()
        self.fc1 = nn.Linear(state_dim, 128)
        self.fc2 = nn.Linear(128, 1)

    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = self.fc2(x)
        return x

# Initialize Actor and Critic
state_dim = env.observation_space.shape[0]
action_dim = env.action_space.n
actor = Actor(state_dim, action_dim)
critic = Critic(state_dim)

# Optimizers
actor_optimizer = optim.Adam(actor.parameters(), lr=actor_lr)
critic_optimizer = optim.Adam(critic.parameters(), lr=critic_lr)

# Experience Buffer
class ExperienceBuffer:
    def __init__(self):
        self.states = []
        self.actions = []
        self.rewards = []
        self.next_states = []
        self.dones = []

    def add_experience(self, state, action, reward, next_state, done):
        self.states.append(state)
        self.actions.append(action)
        self.rewards.append(reward)
        self.next_states.append(next_state)
        self.dones.append(done)

    def sample_experience(self, batch_size):
        indices = np.random.choice(len(self.states), batch_size)
        states = np.array(self.states)[indices]
        actions = np.array(self.actions)[indices]
        rewards = np.array(self.rewards)[indices]
        next_states = np.array(self.next_states)[indices]
        dones = np.array(self.dones)[indices]
        return states, actions, rewards, next_states, dones

# Training Loop
experience_buffer = ExperienceBuffer()
for episode in tqdm(range(max_episodes)):
    state, _info = env.reset()
    episode_reward = 0
    for step in range(max_steps):
        # Select action
        state_tensor = torch.tensor(state, dtype=torch.float32)
        action_logits = actor(state_tensor)
        action_distribution = Categorical(logits=action_logits)
        action = action_distribution.sample().item()

        # Take action
        next_state, reward, terminated, truncated, _info = env.step(action)
        done = terminated or truncated
        episode_reward += reward

        # Add experience to buffer
        experience_buffer.add_experience(state, action, reward, next_state, done)

        # Update state
        state = next_state

        # Check if episode is done
        if done:
            break

    # Sample experience from buffer
    states, actions, rewards, next_states, dones = experience_buffer.sample_experience(batch_size)

    # Compute advantages
    advantages = []
    for i in range(batch_size):
        state_tensor = torch.tensor(states[i], dtype=torch.float32)
        next_state_tensor = torch.tensor(next_states[i], dtype=torch.float32)
        reward = rewards[i]
        done = dones[i]
        value = critic(state_tensor).item()
        next_value = critic(next_state_tensor).item()
        advantage = reward + gamma * next_value * (1 - done) - value
        advantages.append(advantage)
    advantages = np.array(advantages)

    # Compute clipped surrogate objective
    old_action_logits = actor(torch.tensor(states, dtype=torch.float32)).detach()
    old_action_distribution = Categorical(logits=old_action_logits)
    old_action_log_probs = old_action_distribution.log_prob(torch.tensor(actions))
    new_action_logits = actor(torch.tensor(states, dtype=torch.float32))
    new_action_distribution = Categorical(logits=new_action_logits)
    new_action_log_probs = new_action_distribution.log_prob(torch.tensor(actions))
    ratio = torch.exp(new_action_log_probs - old_action_log_probs)
    clipped_ratio = torch.clamp(ratio, 1 - clip_range, 1 + clip_range)
    clipped_surrogate_objective = torch.mean(torch.minimum(ratio * torch.tensor(advantages), clipped_ratio * torch.tensor(advantages)))

        # Compute value loss
    rewards_tensor = torch.tensor(rewards, dtype=torch.float32)
    dones_tensor = torch.tensor(dones, dtype=torch.float32)
    next_values = critic(torch.tensor(next_states, dtype=torch.float32)).detach()
    target_values = rewards_tensor + gamma * next_values * (1 - dones_tensor)
    current_values = critic(torch.tensor(states, dtype=torch.float32))
    value_loss = torch.mean((current_values - target_values) ** 2)

    # Compute entropy loss
    entropy_loss = torch.mean(new_action_distribution.entropy())

    # Compute total loss
    total_loss = -clipped_surrogate_objective + c1 * value_loss - c2 * entropy_loss

    # Backpropagate and update
    actor_optimizer.zero_grad()
    critic_optimizer.zero_grad()
    total_loss.backward()
    actor_optimizer.step()
    critic_optimizer.step()

    # Print episode reward
    print(f'Episode {episode+1}, Reward: {episode_reward}')

# Test the trained agent
state, _info = env.reset()
episode_reward = 0
while True:
    state_tensor = torch.tensor(state, dtype=torch.float32)
    action_logits = actor(state_tensor)
    action_distribution = Categorical(logits=action_logits)
    action = action_distribution.sample().item()
    next_state, reward, terminated, truncated, _info = env.step(action)
    done = terminated or truncated
    episode_reward += reward
    state = next_state
    if done:
        break
print(f'Test Episode Reward: {episode_reward}')

  0%|          | 1/1000 [00:00<11:45,  1.42it/s]

Episode 1, Reward: 23.0


  0%|          | 2/1000 [00:01<12:28,  1.33it/s]

Episode 2, Reward: 36.0


  0%|          | 3/1000 [00:01<10:15,  1.62it/s]

...

100%|█████████▉| 999/1000 [04:01<00:00,  4.26it/s]

Episode 999, Reward: 9.0


100%|██████████| 1000/1000 [04:02<00:00,  4.13it/s]

Episode 1000, Reward: 8.0


Test Episode Reward: 10.0
